In [ ]:
import os
import sys

os.chdir("../..")
sys.path.append("../../")

# Simulate datasets with cell library size

## Introduction

In this example, we will show how to use pyscDesign3 to simulate datasets adjusted by cell library size. The purpose of this example is to show that including the library size when modeling the marginal distribution for each gene can help cells in the synthetic data have more similar library sizes as the cells in the real data.

## Import packages and Read in data

### import pacakges

In [ ]:
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import pyscDesign3

### Read in the reference data

The raw data is from the R package `DuoClustering2018` which contain a set of datasets with true cell type labels and converted to `.h5ad` file using the R package `sceasy`.

In [ ]:
data = ad.read_h5ad("data/Zhengmix4eq.h5ad")
data.obs["cell_type"] = data.obs["phenoid"]
data.obs["cell_type"] = data.obs["cell_type"].astype("category")

We then calculate the library size for each cell.

In [ ]:
sc.pp.calculate_qc_metrics(data,inplace=True)
data.obs.rename(columns={'total_counts':'library'},inplace=True)

In [ ]:
data

## Simulation

Then, we set the mu_formula as cell_type and offsetted by the cell library size to generate new dataset adjusted by library size. The library size is log-transformed because the link function for $\mu$ of the negative binomial distribution in GAMLSS is **log**.

In [ ]:
test = pyscDesign3.scDesign3(n_cores=3,parallelization="pbmcmapply")
simu_res = test.scdesign3(    
    anndata = data,
    default_assay_name = "counts",
    celltype = "cell_type",
    other_covariates = "library",
    mu_formula = "cell_type + offset(log(library))",
    sigma_formula = "1",
    family_use = "nb",
    usebam = True,
    corr_formula = "1",
    copula = "gaussian",
    important_feature = "auto")

Then we can construct new data using the simulated count matrix.

In [ ]:
simu_data = ad.AnnData(X=simu_res["new_count"], obs=simu_res["new_covariate"])
simu_data.layers["log_transformed"] = np.log1p(simu_data.X)
data.layers["log_transformed"] = np.log1p(data.X)

## Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plot = pyscDesign3.plot_reduceddim(
    ref_anndata=data,
    anndata_list=simu_data,
    name_list=["Reference", "scDesign3"],
    assay_use="log_transformed",
    if_plot=True,
    color_by="cell_type",
    n_pc=20,
    point_size=5,
)

In [ ]:
plot["p_umap"]

The violin plot below shows the cells in simulated dataset have similar library size as the cells in the reference dataset.

In [ ]:
sc.pp.calculate_qc_metrics(simu_data,inplace=True)
simu_data.obs.rename(columns={'total_counts':'simu_library'},inplace=True)
df = pd.concat([data.obs["library"],simu_data.obs["simu_library"]],axis=1)

# plot
sns.violinplot(df)
plt.xlabel("Method")
plt.ylabel("library")
plt.xticks(ticks = [0, 1], labels = ["Reference","scDesign3"])
plt.show()